In [1]:
import pandas as pd
import pdb
from bs4 import BeautifulSoup
import logging

In [2]:
#set-up log
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='processor.log', mode='a')
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [ ]:
#set-up output file for card supplies

#set-up output file for features - i.e. number of cards at the end of each round


In [3]:
dir_name = 'DominionPlayerLogs/2012-all/20121231'
#html_doc = 'game-20121231-000041-13409c79.html'
html_doc = 'game-20121231-002653-e5872fde.html'
#html_doc = 'game-20121231-000155-2795fdc1.html'

logging.debug("Processing file: " + str(dir_name + '/' + html_doc))

In [4]:
file = open(dir_name + '/' + html_doc, mode = 'r')
soup = BeautifulSoup(file, 'html.parser')

print(soup.prettify())

<html>
 <head>
  <link href="/semistatic/log.css" rel="stylesheet"/>
  <title>
   Dominion Game #259154
  </title>
 </head>
 <body>
  <pre>Intense wins!
<span class="card-curse">Curses</span>, <span class="card-none">Crossroads</span>, and <span class="card-treasure">Ill-Gotten Gains</span> are all gone.

cards in supply: <span cardname="Colony" class="card-victory">Colony</span>, <span cardname="Conspirator" class="card-none">Conspirator</span>, <span cardname="Council Room" class="card-none">Council Room</span>, <span cardname="Crossroads" class="card-none">Crossroads</span>, <span cardname="Embargo" class="card-none">Embargo</span>, <span cardname="Forge" class="card-none">Forge</span>, <span cardname="Ill-Gotten Gains" class="card-treasure">Ill-Gotten Gains</span>, <span cardname="Militia" class="card-none">Militia</span>, <span cardname="Platinum" class="card-treasure">Platinum</span>, <span cardname="Scout" class="card-none">Scout</span>, <span cardname="Spy" class="card-none">Sp

In [5]:
#convert html contents into a list of tabs, navigable strings etc
pre_tag = soup.find('pre')
pre_tag.contents
print(pre_tag.contents)

['Intense wins!\n', <span class="card-curse">Curses</span>, ', ', <span class="card-none">Crossroads</span>, ', and ', <span class="card-treasure">Ill-Gotten Gains</span>, ' are all gone.\n\ncards in supply: ', <span cardname="Colony" class="card-victory">Colony</span>, ', ', <span cardname="Conspirator" class="card-none">Conspirator</span>, ', ', <span cardname="Council Room" class="card-none">Council Room</span>, ', ', <span cardname="Crossroads" class="card-none">Crossroads</span>, ', ', <span cardname="Embargo" class="card-none">Embargo</span>, ', ', <span cardname="Forge" class="card-none">Forge</span>, ', ', <span cardname="Ill-Gotten Gains" class="card-treasure">Ill-Gotten Gains</span>, ', ', <span cardname="Militia" class="card-none">Militia</span>, ', ', <span cardname="Platinum" class="card-treasure">Platinum</span>, ', ', <span cardname="Scout" class="card-none">Scout</span>, ', ', <span cardname="Spy" class="card-none">Spy</span>, ', and ', <span cardname="Woodcutter" class

In [6]:
#first extract number of players and final score
#look for entries of the form #n <name>: x points' - these are tagged with a 'b'
players = []
scores = []
list_b_tags = soup.find_all('b');
for tag in list_b_tags:
    s = tag.text
    if '#' in s:
        #extract name (assume string of form '#n ' comes before the name)
        players.append(s[3:])
        #next sibing contains the score, assume score is after a string of the form ': ' and is 2 digits long
        score_text = tag.next_sibling
        scores.append(score_text[2:4])
print(players)
print(scores)


['Intense', 'faulkner']
['7 ', '2 ']


In [37]:
#following function parses a single line which consists of number of cards (unless equal to one) and card types
#each separated by a colon and ended with a full stop or a horizontal dashed line. It returns a list of cards 
#with card names duplicated according to the number of them in the row. Also the index of the full stop or 
#dashed line will be returned
def parse_row_into_cards(contents_list):
    card_list = []
    for (k,r) in enumerate(contents_list):
        #Need to be careful that the first content item doesn't contain a '....', causing 
        #the code to pickup a full stop
        if ((('.' not in r) or (k == 0)) and r.name == None and ('----------------------' not in r)):
            #check to see if trashing text contains the number of cards trashed
            #number is contained in last two digits of text
            try:
                no_cards = int(r[-2:])
            except ValueError:
                no_cards = 1
            #take next entry along which should be the card name
            card_type = r.next_sibling.text
            #and add copies of that to the dictionary
            for count in range(0,no_cards):
                card_list.append(card_type)
        elif (('.' in r) or ('----------------------' in r)) and (k != 0):
            return (k, card_list)

In [39]:
#next extract kingdom cards - first we deal with the case where the default set is chosen and players don't 
#make changes

#iterate through contents list under we find an element which contains 'cards in supply'. 
default_supply_cards_start_index = 0
for i, s in enumerate(pre_tag.contents):
    if 'cards in supply' in s:
        default_supply_cards_start_index = i
        break

#there is an initial default set of supply cards. This may be modified later by player choice if this default 
#selection is not chosen. So first determine if the players use the default set
default_supply_set = False
for s in pre_tag.contents[default_supply_cards_start_index:]:
    if 'Default card selection was used' in s:
        default_supply_set = True
        break
    elif '----------------------' in s:
        break
       
#if the default set was used then grab card names
supply_cards = [] 
if default_supply_set == True:
    (index, cards) = parse_row_into_cards(pre_tag.contents[default_supply_cards_start_index:])
    supply_cards = cards
    
print("Default supply cards chosen: " + str(default_supply_set))                
print(len(supply_cards))
print(supply_cards)

Default supply cards chosen: False
0
[]


In [41]:
#next we extract supply cards when players make a choice. In this case we look for the substring 'chosen cards are'
#which tags the start of the set of chosen supply cards. This should be followed by a number of veto statements
#dependent on the number of players
index_end = 0 
if default_supply_set == False:
    for (i,s) in enumerate(pre_tag.contents[default_supply_cards_start_index:]):
        if 'chosen cards are' in s:
            #loop over chosen supply cards
            (index, cards) = parse_row_into_cards(pre_tag.contents[default_supply_cards_start_index + i + 1:])
            index_end = default_supply_cards_start_index + i + index + 1
            supply_cards = cards

#next each player can veto a card
vetoed_cards = []
for player in players:
    for s in pre_tag.contents[index_end:]:
        check_string = player + ' vetoes'
        if check_string in s:
            vetoed_cards.append(s.next_sibling.text)

#remove vetoed cards from list of supply cards
supply_cards = list(filter(lambda x: x not in vetoed_cards, supply_cards))

print(default_supply_cards_start_index)
print(vetoed_cards)             
print(supply_cards)

6
['Scrying Pool', 'Black Market']
['Conspirator', 'Council Room', 'Crossroads', 'Embargo', 'Forge', 'Ill-Gotten Gains', 'Militia', 'Scout', 'Spy', 'Woodcutter']


In [9]:
#next loop through the turns and extract and bought or gained cards

#use a dictionary of dictionaries to track gained cards by turn by player
gained_cards_by_turn = {}
for player in players:
    incremental_cards_by_turn[player] = {}
    incremental_cards_by_turn[player]['buys'] = {}
    incremental_cards_by_turn[player]['trashing'] = {}
    incremental_cards_by_turn[player]['gaining'] = {}
    incremental_cards_by_turn[player]['trashes'] = {}
    incremental_cards_by_turn[player]['gains'] = {}
    turn_counter = 1
    #string to check for a buy action
    check_buy = player + ' buys'
    #string to check if current player is trashing a card
    check_trashing = 'trashing'
    #string to check if current player is gaining a card
    check_gaining = 'gaining'
    #string to check if another player trashes a card. Is 'trashes' used instead of 
    #'trashing' when done out of turn?
    check_another_player_trashes = [x + ' trashes' for x in players] # need to check this!
    #string to check if another player is gains a card (gains is used rather than
    #gaining when a player gets a card out of turn)
    check_another_player_gains = [ x + ' gains' for x in players]
    for (i,p) in enumerate(pre_tag.contents[index_end:]):
        check_turn = player + '\'s' + ' turn ' + str(turn_counter)
        if check_turn in p:
            buy_card_list = []
            trashing_card_list = []
            gaining_card_list = []
            gains_card_list = {} #needs to be a dictionary as multiple players may gain in another player's round
            trashes_card_list = {} #needs to be a dictionary as multiple players may need to trash in another player's round
            for (j,q) in enumerate(pre_tag.contents[index_end + i:]):
                another_player_gains_list = [ x in q for x in check_another_player_gains]
                another_player_trashes_list = [ x in q for x in check_another_player_trashes]
                if check_buy in q:
                    buy_card_list.append(q.next_sibling.text) #what if the player buys more than
                    #one card does this go on a separate line?
                elif check_trashing in q:
                    #Q: can you trash nothing in the log files?
                    for (k,r) in enumerate(pre_tag.contents[index_end + i + j:]):
                        #There is a .... before trashing which gets confused with the '.'
                        #at the end of the line, hence the following check
                        if (('.' not in r) or (k == 0)) and r.name == None:
                            #check to see if trashing text contains the number of cards trashed
                            #number is contained in last two digits of text
                            try:
                                no_cards = int(r[-2:])
                            except ValueError:
                                no_cards = 1
                            #take next entry along which should be the card name
                            card_type = r.next_sibling.text
                            #and add copies of that to the dictionary
                            for count in range(0,no_cards):
                                trashing_card_list.append(card_type)
                        elif '.' in r and check_trashing_string not in r:
                            #note we eneded to check that we are looking for the full stop at the
                            #end of the line and not in the '... trashing ' string
                            break
                elif check_gaining in q:
                    for (k,r) in enumerate(pre_tag.contents[index_end + i + j:]):
                        #There are two possibilities here, either gaining a card which
                        #has the format 'gaining a Province.' say or also 
                        #'gaining a Copper in the hand.'. Again is a .... before gaining which gets confused 
                        #with the '.'at the end of the line.
                        #also we need to check for a 'gaining nothing' case
                        if 'gaining nothing' in r:
                            break
                        elif (('.' not in r) or (k == 0)) and r.name == None:
                            #check to see if trashing text contains the number of cards trashed
                            #number is contained in last two digits of text
                            try:
                                no_cards = int(r[-2:])
                            except ValueError:
                                no_cards = 1
                            #take next entry along which should be the card name
                            card_type = r.next_sibling.text
                            #and add copies of that to the dictionary
                            for count in range(0,no_cards):
                                gaining_card_list.append(card_type)
                        elif '.' in r and check_gaining_string not in r:
                            #note we needed to check that we are looking for the full stop at the
                            #end of the line and not in the '... gaining ' string
                            break
                    #pdb.set_trace()
                elif any(another_player_gains_list):
                    #next we check if an opponent gains a card, e.g. a curse card
                    #need to loop through opponents
                    for (index, player_gains_check) in enumerate(another_player_gains_list):
                        if player_gains_check == true:
                            player = players[index]
                            
                            
                    
                    #pdb.set_trace()
                elif any(another_player_trashes_list):
                    z = 0
                    #pdb.set_trace()
                elif 'draws' in q.text:
                    break;
                    incremental_cards_by_turn[player]['buys'] = buy_card_list
                    incremental_cards_by_turn[player]['trashing'] = trashing_card_list
                    incremental_cards_by_turn[player]['gaining'] = gaining_card_list
                    incremental_cards_by_turn[opp_player]['trashes'] = trashes_card_list
                    incremental_cards_by_turn[opp_player]['gains'] = gains_card_list
                    turn_counter += 1
                    break
print(gained_cards_by_turn)          

{'Intense': {'buys': {1: ['Silver'], 2: ['Militia'], 3: ['Ill-Gotten Gains'], 4: ['Silver'], 5: ['Ill-Gotten Gains'], 6: ['Silver'], 7: ['Ill-Gotten Gains'], 8: ['Ill-Gotten Gains'], 9: ['Ill-Gotten Gains'], 10: ['Gold'], 11: ['Crossroads'], 12: ['Platinum'], 13: ['Forge'], 14: ['Council Room'], 15: ['Spy'], 16: ['Silver'], 17: [], 18: ['Silver'], 19: [], 20: ['Forge'], 21: ['Spy'], 22: ['Forge'], 23: [], 24: ['Council Room'], 25: ['Platinum'], 26: ['Woodcutter'], 27: ['Council Room'], 28: ['Platinum', 'Woodcutter'], 29: ['Crossroads'], 30: [], 31: ['Forge']}, 'trashes': {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: ['Curses', 'Curses', 'Coppers', 'Coppers'], 20: [], 21: [], 22: [], 23: ['Coppers', 'Coppers', 'Coppers', 'Ill-Gotten Gains'], 24: [], 25: [], 26: ['Curse', 'Council Room', 'Ill-Gotten Gains'], 27: ['Curse', 'Estate', 'Copper'], 28: [], 29: ['Curse', 'Copper', 'Ill-Gotten Gains'], 

In [ ]:
#convert dictionary into output txt file of the format of the features listener file from TAG

#dont forgot to add in starting set of cards

In [ ]:
#print(soup.title)
#print(soup.head)
#print(soup.children)
#children = soup.children
#print(next(children))
#print(soup.get_text())
#soup.find_all('span')